In [1]:
import numpy as np
import pandas as pd
import nltk
# import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emers\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emers\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load data into dataframe
df = pd.read_csv('merged_data.csv')

In [3]:
df.head(10)

,victim_name,text_combine
0,Aaron Brockway,text
1,Aaron Brockway,R.I.P Aaron Brockway Im lighting this one fo...
2,Aaron Brockway,Rip. Aaron Brockway 🙏
3,Aaron Carlos Martinez,text
4,Aaron Carlos Martinez,Aaron Carlos Martinez 18 : #IHateTimWaterman
5,Aaron Carlos Martinez,1st three listed RT @Scottelands36: 3 keepers...
6,Aaron Christopher Scott,text
7,Aaron Christopher Scott,Christopher Eccleston Domhnall Gleeson Aaro...
8,Aaron Harts,text
9,Aaron Harts,Check it out! I donated to My son Aaron Harts...


In [4]:
# Remove missing values and the tweets with just " text"
d = df.dropna(subset=['text_combine'])
d['text_combine'] = df['text_combine'].apply(lambda x: str(x) if isinstance(x, str) else '') # checks if tweet is string
d.drop(d[d['text_combine'] == ' text'].index, inplace=True) # removes tweets with just ' text'
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609275 entries, 1 to 611509
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   victim_name   609275 non-null  object
 1   text_combine  609275 non-null  object
dtypes: object(2)
memory usage: 13.9+ MB


C:\Users\emers\AppData\Local\Temp\ipykernel_632\2462410335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['text_combine'] = df['text_combine'].apply(lambda x: str(x) if isinstance(x, str) else '') # checks if tweet is string
C:\Users\emers\AppData\Local\Temp\ipykernel_632\2462410335.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.drop(d[d['text_combine'] == ' text'].index, inplace=True) # removes tweets with just ' text'


In [5]:
d.reset_index(inplace=True, drop=True)

In [6]:
d.head(10)

,victim_name,text_combine
0,Aaron Brockway,R.I.P Aaron Brockway Im lighting this one fo...
1,Aaron Brockway,Rip. Aaron Brockway 🙏
2,Aaron Carlos Martinez,Aaron Carlos Martinez 18 : #IHateTimWaterman
3,Aaron Carlos Martinez,1st three listed RT @Scottelands36: 3 keepers...
4,Aaron Christopher Scott,Christopher Eccleston Domhnall Gleeson Aaro...
5,Aaron Harts,Check it out! I donated to My son Aaron Harts...
6,Aaron Harts,Check it out! I donated to My son Aaron Harts...
7,Aaron James Pritchard,Aaron James Pritchard 21 : #IHateTimWaterman
8,Aaron James Pritchard,Aaron James Pritchard 21 https://t.co/BIWiKo...
9,Aaron James Pritchard,Aaron James Pritchard 21: Aaron James Pritc...


In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stopwords_EN = nltk.corpus.stopwords.words('english')
stopwords_SPA = nltk.corpus.stopwords.words('spanish')
stopwords = stopwords_EN + stopwords_SPA
stopwords += ['https', 'rt', 'http']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emers\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emers\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# includes english and spanish stopwords as well as "https" and "rt" (retweet)
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [9]:
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")


# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        # print(token)
        if token == 'r.i.p' or token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [10]:
text = d.loc[:, 'text_combine'].tolist()

In [11]:
text[:10]

[' R.I.P Aaron Brockway  Im lighting this one for you',
 ' Rip. Aaron Brockway 🙏',
 ' Aaron Carlos Martinez  18 : #IHateTimWaterman',
 ' 1st three listed RT @Scottelands36: 3 keepers AL only Roto 5x5. Price $23  JD Martinez $12  Salazar $6  Aaron Hicks $6  Carlos Rodon $6.',
 ' Christopher Eccleston  Domhnall Gleeson  Aaron Paul Wes Bentley  Ben Foster Scott Speedman https://t.co/8Y4Ljqeds3',
 ' Check it out! I donated to My son Aaron Harts final exspensive https://t.co/i6w7GhvlNG',
 ' Check it out! I donated to My son Aaron Harts final exspensive  Murderd walking to 7eleven https://t.co/i6w7GhvlNG',
 ' Aaron James Pritchard  21 : #IHateTimWaterman',
 ' Aaron James Pritchard  21 https://t.co/BIWiKotXaC',
 ' Aaron James Pritchard  21:  Aaron James Pritchard  a 21-year-old black male  died Monday  May 16  in Carson  ... https://t.co/gVEIEjqKOd']

In [12]:
tokenization_and_stemming(text[6])

['check', 'donat', 'aaron', 'hart', 'final', 'exspens', 'murderd', 'walk']

In [13]:
tokenization_and_stemming(text[2])

['aaron', 'carlo', 'martinez', 'ihatetimwaterman']

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(text) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

C:\Users\emers\anaconda3\envs\Causal_Pathways\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\emers\anaconda3\envs\Causal_Pathways\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft

In total, there are 609275 reviews and 99 terms.


In [15]:
tfidf_matrix

<609275x99 sparse matrix of type '<class 'numpy.float64'>'
	with 1947711 stored elements in Compressed Sparse Row format>

In [16]:
tfidf_matrix[0]

<1x99 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [36]:
tokenization_and_stemming(text[0])

['r.i.p', 'aaron', 'brockway', 'im', 'light', 'one']

In [18]:
# words
tf_selected_words = tfidf_model.get_feature_names_out()
tf_selected_words

array(['abraham', 'ahora', 'alberto', 'alcald', 'alejandro', 'amigo',
       'amp', 'andré', 'antonio', 'asesinado', 'año', 'brown', 'caicedo',
       'carlo', 'castillo', 'cortez', 'cruz', 'cárdena', 'da', 'danc',
       'daniel', 'david', 'dedic', 'deputi', 'director', 'día', 'díaz',
       'father', 'fernando', 'flore', 'garcia', 'garcía', 'gonzález',
       'hace', 'hoy', 'instructor', 'jair', 'jean', 'jesú', 'jorg',
       'jose', 'josé', 'juan', 'julianroman', 'julio', 'kill', 'leyder',
       'león', 'lui', 'lópez', 'manuel', 'martinez', 'martín', 'martínez',
       'mass', 'mendoza', 'miguel', 'ming', 'montaño', 'monterey',
       'moreno', 'ndelriego', 'omar', 'oscar', 'pablo', 'park', 'paul',
       'pedro', 'perlaza', 'pimentel', 'president', 'pérez', 'ramo',
       'ramírez', 'ramón', 'richard', 'roberto', 'rodríguez', 'ryan',
       'san', 'sergio', 'shoot', 'shot', 'si', 'silva', 'smith', 'soto',
       'sánchez', 'thread', 'time', 'torr', 'twyman', 'valencia',
       've

In [19]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5)

# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)

## 5 topics

In [20]:
print(lda_output.shape)
print(lda_output)

(609275, 5)
[[0.2        0.2        0.2        0.2        0.2       ]
 [0.2        0.2        0.2        0.2        0.2       ]
 [0.0856267  0.33636404 0.08324808 0.08324809 0.4115131 ]
 ...
 [0.59883139 0.1        0.1        0.1011686  0.1       ]
 [0.2        0.2        0.2        0.2        0.2       ]
 [0.2        0.2        0.2        0.2        0.2       ]]


In [21]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(text))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)  #df.values():返回数组形式的数据
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.20,0.20,0.20,0.20,0.20,0
Doc1,0.20,0.20,0.20,0.20,0.20,0
Doc2,0.09,0.34,0.08,0.08,0.41,4
Doc3,0.09,0.34,0.08,0.08,0.41,4
Doc4,0.10,0.10,0.60,0.10,0.10,2
Doc5,0.20,0.20,0.20,0.20,0.20,0
Doc6,0.20,0.20,0.20,0.20,0.20,0
Doc7,0.20,0.20,0.20,0.20,0.20,0
Doc8,0.20,0.20,0.20,0.20,0.20,0
Doc9,0.20,0.20,0.20,0.20,0.20,0


In [22]:
df_document_topic['topic'].value_counts().to_frame()

,topic
0,246986
2,109723
1,98114
3,82288
4,72164


In [23]:
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names_out()
df_topic_words.index = topic_names

df_topic_words.head()

,abraham,ahora,alberto,alcald,alejandro,amigo,amp,andré,antonio,asesinado,...,time,torr,twyman,valencia,venezuela,victim,wei,álvaro,ángel,óscar
Topic0,0.200093,0.201659,0.895665,0.200642,0.227416,0.200999,5706.479445,0.201438,8534.472836,2815.818265,...,0.200268,0.201207,0.200107,3394.782004,0.200442,0.200064,0.200022,220.549192,0.200911,0.201015
Topic1,3987.619897,15.435655,4693.465908,0.200688,0.263959,0.200440,0.200455,730.754590,0.201273,1915.974707,...,0.200107,0.209794,0.200012,0.200485,0.200551,0.200024,0.200009,528.910412,0.200805,2776.525043
Topic2,0.200022,4936.230934,0.200206,0.200089,0.200219,0.200197,0.200125,15241.114718,0.200116,0.200566,...,0.200065,0.200321,0.200007,0.200207,0.200063,0.200016,0.200006,3060.435399,0.200125,0.200119
Topic3,0.200055,0.216322,0.201114,3895.571730,0.211114,3605.353847,0.233815,0.200626,0.200867,0.204893,...,4448.757040,5235.467949,3260.491219,1748.754276,5431.830150,0.200096,0.200013,0.501365,0.200732,1336.851315
Topic4,0.200032,0.200901,0.200382,0.200096,5865.062721,0.200203,4932.516327,0.200365,0.200341,0.200106,...,0.200190,0.200238,0.200019,0.200130,0.200090,6199.960609,5593.106879,0.203198,11775.356587,0.200143


In [24]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names_out())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words)) #array.take(): 从给出的index取出数据
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,lui,garcía,josé,martínez,antonio,garcia,julio,roberto,richard,pablo,amp,kill,mendoza,president,valencia
Topic 1,carlo,daniel,pérez,lópez,gonzález,fernando,martín,ramírez,josé,da,ramón,castillo,juan,ramo,alberto
Topic 2,año,manuel,juan,andré,paul,jean,montaño,jair,cortez,perlaza,cárdena,leyder,silva,david,si
Topic 3,hoy,jesú,sánchez,shot,san,flore,venezuela,jose,torr,father,día,director,time,deputi,ryan
Topic 4,miguel,ángel,rodríguez,shoot,park,victim,danc,alejandro,monterey,pedro,ming,wei,omar,smith,brown


## 10

In [25]:
lda_10 = LatentDirichletAllocation(n_components = 10)

# document topic matrix for tfidf_matrix_lda
lda_output = lda_10.fit_transform(tfidf_matrix)

In [26]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda_10.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(text))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)  #df.values():返回数组形式的数据
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,topic
Doc0,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0
Doc1,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0
Doc2,0.04,0.04,0.04,0.30,0.04,0.04,0.04,0.37,0.04,0.04,7
Doc3,0.04,0.04,0.04,0.30,0.04,0.04,0.04,0.37,0.04,0.04,7
Doc4,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.55,9
Doc5,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0
Doc6,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0
Doc7,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0
Doc8,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0
Doc9,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0


In [27]:
df_document_topic['topic'].value_counts().to_frame()

,topic
0,189977
9,76930
7,58285
3,49075
1,45671
2,45134
8,40663
5,39388
6,36963
4,27189


In [28]:
# topic-word matrix
df_topic_words = pd.DataFrame(lda_10.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names_out()
df_topic_words.index = topic_names

df_topic_words

,abraham,ahora,alberto,alcald,alejandro,amigo,amp,andré,antonio,asesinado,...,time,torr,twyman,valencia,venezuela,victim,wei,álvaro,ángel,óscar
Topic0,0.100000,0.100012,0.100005,0.100007,0.100012,0.100005,0.100002,0.100006,0.708308,0.100002,...,0.100001,0.100003,0.100000,0.100003,0.100002,0.100000,0.100000,260.235936,0.100002,0.100002
Topic1,0.100000,0.100004,4694.063256,0.100002,1036.707524,0.100003,0.100001,0.100004,0.100004,744.745555,...,0.100000,4638.931662,0.100000,0.100002,0.100013,0.100000,0.100000,22.145279,0.100003,1175.951577
Topic2,0.100001,0.100001,0.100004,0.100001,0.104117,0.100001,111.073837,1596.310064,0.101478,487.069870,...,0.100001,596.547827,0.100000,0.100002,5431.731268,0.100001,0.100000,0.100002,0.100001,0.100001
Topic3,0.100000,0.100004,0.100003,0.100002,0.101168,0.100004,0.100001,0.111373,7059.500168,2220.000301,...,0.100000,0.100006,0.100000,0.100001,0.100001,0.100000,0.100000,0.153914,0.100001,0.100003
Topic4,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,10460.858559,0.100000,0.100000,0.100000,...,0.100001,0.100000,0.100000,0.100000,0.100000,6199.860807,5593.006929,0.100000,0.100000,0.100000
Topic5,0.100000,0.100005,0.100002,0.100002,0.100003,0.100002,52.297847,0.100007,1474.365464,0.100000,...,0.100000,0.100002,0.100000,5143.237089,0.100001,0.100000,0.100000,526.430162,11775.259150,0.100001
Topic6,3987.520098,0.100002,0.100002,0.100005,0.100007,0.100001,0.100002,0.100006,0.100003,1279.982803,...,0.100000,0.100002,0.100000,0.100001,0.100008,0.100000,0.100000,0.100002,0.100001,2937.226045
Topic7,0.100000,4951.385442,0.100003,0.100005,4828.452590,0.100002,0.114064,0.100500,0.100005,0.100001,...,0.100002,0.100005,0.100000,0.100002,0.100002,0.100000,0.100000,0.100005,0.100001,0.100003
Topic8,0.100000,0.100002,0.100002,3895.473220,0.100008,3605.255667,14.785855,0.100002,0.100002,0.100002,...,4448.657665,0.100003,3260.391363,0.100001,0.100001,0.100000,0.100000,0.100004,0.100001,0.100004
Topic9,0.100000,0.100000,0.100000,0.100000,0.100000,0.100001,0.100000,14375.349774,0.100000,0.100002,...,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,3001.134262,0.100000,0.100000


In [29]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names_out())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words)) #array.take(): 从给出的index取出数据
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda_10, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,garcía,lui,pablo,martín,moreno,jose,josé,juan,manuel,si,álvaro,pedro,antonio,carlo,pérez
Topic 1,hoy,gonzález,león,fernando,alberto,torr,día,josé,año,ramón,martínez,óscar,alejandro,san,lui
Topic 2,garcia,kill,ramo,flore,omar,venezuela,sergio,director,shot,andré,lui,mendoza,torr,asesinado,pedro
Topic 3,carlo,lópez,martínez,julio,antonio,si,juan,ramón,josé,asesinado,pérez,lui,manuel,president,martinez
Topic 4,amp,shoot,park,victim,danc,monterey,ming,wei,mass,thread,dedic,instructor,ndelriego,kill,shot
Topic 5,miguel,ángel,rodríguez,valencia,san,mendoza,josé,antonio,juan,álvaro,lui,año,amp,manuel,jesú
Topic 6,daniel,josé,pérez,oscar,cruz,abraham,hace,soto,díaz,óscar,pimentel,año,hoy,asesinado,ramírez
Topic 7,david,silva,smith,roberto,richard,jesú,castillo,brown,ahora,jorg,alejandro,martinez,josé,juan,moreno
Topic 8,pedro,president,sánchez,da,ramírez,father,time,deputi,ryan,alcald,amigo,twyman,shot,oscar,martinez
Topic 9,año,manuel,andré,paul,jean,montaño,jair,cortez,perlaza,juan,cárdena,leyder,álvaro,julianroman,caicedo


##15

In [30]:
lda_15 = LatentDirichletAllocation(n_components = 15)

# document topic matrix for tfidf_matrix_lda
lda_output = lda_15.fit_transform(tfidf_matrix)

In [31]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda_15.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(text))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)  #df.values():返回数组形式的数据
df_document_topic['topic'] = topic

df_document_topic.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,topic
Doc0,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0
Doc1,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0
Doc2,0.03,0.03,0.03,0.03,0.03,0.28,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.36,14
Doc3,0.03,0.03,0.03,0.03,0.03,0.28,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.36,14
Doc4,0.03,0.03,0.03,0.03,0.53,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,4


In [32]:
df_document_topic['topic'].value_counts().to_frame()

,topic
0,177026
4,66453
14,46971
12,43405
5,39595
8,36706
9,32551
11,28561
2,27219
6,21861


In [33]:
# topic-word matrix
df_topic_words = pd.DataFrame(lda_15.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names_out()
df_topic_words.index = topic_names

df_topic_words

,abraham,ahora,alberto,alcald,alejandro,amigo,amp,andré,antonio,asesinado,...,time,torr,twyman,valencia,venezuela,victim,wei,álvaro,ángel,óscar
Topic0,3987.486767,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,2.618417,427.818376,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,528.212516
Topic1,0.066667,0.066667,0.066667,0.066667,5865.032095,0.066667,0.066667,679.930489,1181.758044,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
Topic2,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,10249.548143,0.066667,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,6199.827475,5592.973596,0.066667,0.066667,0.066667
Topic3,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,852.628626,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,3584.898453
Topic4,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,12860.070863,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,357.997227,0.066667,0.066667
Topic5,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,3451.146710,...,0.066667,5235.274628,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
Topic6,0.066667,0.066667,4694.029942,0.066667,0.066667,0.066667,389.214709,0.066667,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
Topic7,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,49.626891,0.174398,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667
Topic8,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,1121.034132,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,250.357160,11775.225827,0.066667
Topic9,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,0.066667,...,0.066667,0.066667,0.066667,0.066667,5431.586114,0.066667,0.066667,0.066667,0.066667,0.066667


In [34]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names_out())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words)) #array.take(): 从给出的index取出数据
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda_15, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,josé,león,oscar,abraham,soto,díaz,pimentel,daniel,pérez,hace,hoy,año,juan,óscar,jose
Topic 1,martínez,alejandro,ramo,sergio,david,antonio,sánchez,pedro,andré,josé,manuel,díaz,juan,lui,pérez
Topic 2,amp,shoot,park,victim,danc,monterey,ming,wei,mass,thread,dedic,instructor,ndelriego,shot,kill
Topic 3,pérez,roberto,president,óscar,mendoza,hoy,año,lópez,asesinado,lui,juan,manuel,daniel,carlo,sánchez
Topic 4,año,manuel,andré,paul,jean,jair,montaño,cortez,cárdena,perlaza,leyder,juan,julianroman,álvaro,hoy
Topic 5,carlo,si,juan,garcia,martín,torr,asesinado,lui,garcía,julio,pérez,manuel,josé,san,mendoza
Topic 6,richard,pablo,rodríguez,alberto,garcía,lui,juan,mendoza,martinez,amp,manuel,josé,flore,carlo,sánchez
Topic 7,gonzález,kill,daniel,smith,hace,shot,año,sánchez,andré,juan,carlo,antonio,shoot,lui,amp
Topic 8,miguel,ángel,silva,david,rodríguez,garcia,da,andré,jesú,álvaro,juan,mendoza,josé,manuel,díaz
Topic 9,fernando,julio,jesú,venezuela,ramírez,director,cruz,martínez,josé,ramo,ramón,lui,año,juan,manuel
